In [500]:
#Notes:
#Shortned training is what was actually used for finetuning GPT-3

'C:\\Users\\Ali\\Documents\\Auto Hanson'

In [1]:
#Set the seed to make things replicable
from bs4 import BeautifulSoup
import requests
import re
import os
import unicodedata
import json
import random
random.seed(0)
path = os.getcwd()
path

'C:\\Users\\Ali\\Documents\\Auto Hanson'

In [253]:
#Use this to turn a string into an acceptable substring of a filename
def slugify(value, allow_unicode=False):
    """
    Taken from https://github.com/django/django/blob/master/django/utils/text.py
    Convert to ASCII if 'allow_unicode' is False. Convert spaces or repeated
    dashes to single dashes. Remove characters that aren't alphanumerics,
    underscores, or hyphens. Convert to lowercase. Also strip leading and
    trailing whitespace, dashes, and underscores.
    """
    value = str(value)
    if allow_unicode:
        value = unicodedata.normalize('NFKC', value)
    else:
        value = unicodedata.normalize('NFKD', value).encode('ascii', 'ignore').decode('ascii')
    value = re.sub(r'[^\w\s-]', '', value.lower())
    return re.sub(r'[-\s]+', '-', value).strip('-_')
slugify('Why Not Impossible Worlds?')

'why-not-impossible-worlds'

In [277]:
#Gets all the URLS for each months posts on OvercomingBias. You need the headers arg as otherwise the page will reject 
#the request. To find such headers, go to developer tools, network, GET and look for a string like below.
def get_month_archives():

    URL = "https://www.overcomingbias.com/archives"
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 6.3; Win64; x64; rv:106.0) Gecko/20100101 Firefox/106.0"}
    page = requests.get(URL, headers=headers)
    soup = BeautifulSoup(page.content, "html.parser")
    month_archives = list(map(lambda x: x.get("href"), soup.find("div", {"id": "monthly-archives"}).find_all("a")))
    return month_archives


In [348]:
def scrape_OB_from_month_archives(month_archives, path):
    for i in range(len(month_archives)):
        month_URL = month_archives[i] 
        print(month_URL)
        ls = get_articles(get_month_links(month_URL))
        save_articles_from_article_list(ls, path)


In [343]:
def save_articles_from_article_list(article_list, path):
    
    
    for article in article_list:
        os.chdir(path+"\\Articles")
        with open(slugify(article["title"])+".txt", "w", encoding="utf-8") as f:
            for k in article:
                f.write(k + ": " +article[k]+"\n\n")
            f.close()
    os.chdir(path)


In [344]:
def get_month_links(month_URL):

    page = 1
    flag = True
    links_page = []
    while flag:
        
        page_n = requests.get(month_URL+"/page/"+str(page), headers=headers)
        
        soup_N = BeautifulSoup(page_n.content, "html.parser")
        

        links_page += list(map(lambda x: x.find("a").get("href"), soup_N.find_all("h2", {"class": "entry-title"})))
        
        try:
            if soup_N.find("div", {"class":"nav-next"}).find("a") != None or soup_N.find("dive", {"class": "nav-next"}) ==None:
                page +=1
        except: 
            flag = False
            
    return links_page

In [338]:
def get_articles(month_urls):

    articles = []
    for link in month_urls:
        article_page = requests.get(link, headers = headers)
        article_soup = BeautifulSoup(article_page.content, "html.parser")

        title, author, date = get_title_author_date(article_soup)
        if (title, author, date ) !=(None, None, None):
            text = article_soup.find("div", {"class": "entry-content"}).get_text()
            text = text.replace("\nGD Star Ratingloading...", "")
            articles.append({"title": title,"author":author, "date":date, "text":text})
    return articles

In [214]:
def get_title_author_date(soup):
    try:
        title = soup.find("h1", {"class":"entry-title"}).get_text()
        author = soup.find("span", {"class":"author vcard"}).find("a").get_text()
        date = soup.find("span", {"class":"entry-date"}).get_text()
    except:
        title, author, date = (None, None, None)
    return (title, author, date)
#get_title_author_date(article_soup)

In [ ]:
with open("classes_cleaned.txt", "r", encoding="utf-8") as f:
    lines = f.readlines()
#temp = lambda txt: clean_txt("!| [Oo]btained|Level |levelled|[0-9]+|.*[\—\-\–].*|.*[Ss]kill.*" ,"", txt)
def temp_1(lines):
    temp = lambda txt: clean_txt(".*Rank  Horror.*|Temporary|.*:.*|[Rr]emoved|Class|.*Boon of the.*|.*\(.*|Aspect.*|\.|My Noble Virtue:.*|Bound Spell:.*|Class Consolidation Failed. Unable to Advance|^\[\\n$|^\[\].*|^\[$|Class Consolidation: |Spell [\—\-\–].*|Condition.*|Conditions Met.*|.*Miracle.*|!| [Oo]btained|Level |levelled|[0-9]+|.*[\—\-\–]{1}\]|.*[Ss]kill.*" ,"", txt)
    remove_extra_spaces = lambda txt: clean_txt("\[ ","[" ,clean_txt(" \]", "]", txt))
    x = [];
    for i in range(len(lines)):
        line = lines[i]
        if temp(line) != "\n":
            x.append(remove_extra_spaces(temp(line)))
    return x
new_lines = list(map(lambda x: x+"\n",  list(set(map( lambda x: x.strip("\n"), temp_1(lines))).difference(spell_set).difference(skill_set))))
with open("classes_cleaned.txt", "w", encoding="UTF-8") as f:
    f.write("".join(new_lines))
f.close()

In [351]:
#Uncoment to download OB hosted articles. 
#If an article redirects to another website, it won't be included.
#scrape_OB_from_month_archives(get_month_archives(), path)

In [10]:
def get_text_dict(txt):
    ls = [ i.split(": ",1) for i in  txt.split("\n\n")]
    k_v_list = []
    for i in ls:
        if len(i)==2:
            k,v = i
            
            k = k.strip("\n").strip(" ")
            k_v_list.append([k,v])
    return  dict(k_v_list)
#assert get_text_dict("text: Hi \n\n Author: me") == {' Author': 'me', 'text': 'Hi '}


In [12]:
seperator = "\n\n###\n\n"

def get_Date_Title_data(txt_dict, seperator, stop_sequence):
    return {"prompt": "Date: " + txt_dict["date"] +seperator, "completion":" " +"Title: " +txt_dict["title"] + stop_sequence}
    
        
def get_DateTitle_Intro_data(txt_dict,chunked_text, seperator, stop_sequence):
    completion = chunked_text[0]    
    if completion[0]!=" ":
        completion = " "+completion
    completion += stop_sequence            
    
    prompt = "Date: " + txt_dict["date"] + " \n" + "Title: " +txt_dict["title"] + seperator
    
    return {"prompt": prompt, "completion": completion}

def get_essay_data(my_dict, chunked_text, seperator, stop_sequence):
    ls = []
    for i in range(len(chunked_text)-1):
        if i ==0:
            prompt = "Date: " + my_dict["date"] + " \n" + "Title: " +my_dict["title"]+ "\n" + chunked_text[i]+seperator
        else:
            prompt = chunked_text[i] + seperator
        completion = " "+chunked_text[i+1]+stop_sequence
        if completion[0]!=" ":
            completion += " "
        ls.append({"prompt": prompt, "completion": completion})
    return ls

    
    
    
def chunk_text(my_dict):
    text = my_dict["text"]
    lines = text.split("\n")
    txt = "";
    intro = "";
    chunked_text = []
    for i in lines:
        if len(txt  +i)>500*4 and intro == "":
            intro = txt
            chunked_text.append(intro)
            txt = ""
            
        elif len(txt+i)>2000*2:
            chunked_text.append(txt)
            txt = ""
        txt+=i
        
    if txt !="":
        chunked_text.append(txt)

    return chunked_text

In [13]:
def write_data_to_destination(path, destination, article_name, sep, stop_seq):
    os.chdir(path+"\Articles")
    with open(article_name, "r", encoding="utf-8") as f:
        text = f.read()
    f.close()
    os.chdir(path)
    text = text.replace("\n\n\n", "\n\n")
    
    my_dict = get_text_dict(text)
    if my_dict["author"].lower() != "robin hanson":
        return 0;
    
    DTd = get_Date_Title_data(my_dict, sep, stop_seq)
    chunked_text = chunk_text(my_dict)
    DTId = get_DateTitle_Intro_data(my_dict,chunked_text,  sep, stop_seq)
    Ed = get_essay_data(my_dict, chunked_text, sep, stop_seq)
    
    
    with open(destination + ".jsonl","a", encoding="utf-8") as f:
        json.dump(DTd, f)
        f.write("\n")
        json.dump(DTId, f)
        f.write("\n")
        for i in Ed:
            json.dump(i, f)
            f.write("\n")
    f.close()
def generate_train_and_val_files_from_article_list(path, article_list, sep, stop_seq):
    for i in article_list:
        try:
            x = random.uniform(0,1)
            if x<0.0125:

                write_data_to_destination(path, "validation", i, sep, stop_seq)
            elif x<0.25:
                write_data_to_destination(path, "training", i, sep, stop_seq)
            else:
                pass
        except Exception as e:
            pass

#If the folder where all the articles are is NOT Articles, then you need to modify this function to chdir to the write folder
#If the folder where all the articles are has non .txt files, you need to modify this function so that articles is a list
#of only .txt files
def generate_training_and_val_data(path, sep, stop_seq):
    os.chdir(path+"\Articles")
    #This is to remove the ipynb checkpoint folder.
    articles = os.listdir()[1:]
    os.chdir(path)
    generate_train_and_val_files_from_article_list(path, articles, sep, stop_seq)

In [14]:
sep = "\n\n###\n\n"
stop_seq = "\n\n@@@\n\n"

random.seed(0)
#IMPORTANT BEFORE YOU RUN THIS, RUN RANDOM.SEED(0) OR ELSE YOU'LL GET A DIFFERENT TRAINING/VAlIDATION SPLIT
#THIS WILL ADD ONTO THE TRAINING AND VALIDATION .jsonl FILES. NOT OVERWRITE THEM.
generate_training_and_val_data(path, sep, stop_seq)

#RUN THE STUFF BELOW TOO

In [2]:
def get_dataset(name_jsonl):
    with open(name_jsonl, "r", encoding = "utf-8") as f:
        lines = f.read().split("\n")
    f.close()
    return list(map( lambda x: json.loads(x), lines[0:-1]))

def print_len_dataset(name_jsonl):
    print(name_jsonl + " has length " + str(len(get_dataset(name_jsonl)))+"\n")

In [3]:
def write_new_dataset(name, dataset):
    with open(name+".jsonl", "w", encoding = "utf-8") as f:
        for i in dataset:
            json.dump(i, f)
            f.write("\n")
        f.close()

In [4]:
training = get_dataset("training.jsonl")
validation = get_dataset("validation.jsonl")
print_len_dataset("training.jsonl")
print_len_dataset("validation.jsonl")

training.jsonl has length 1953

validation.jsonl has length 114



In [5]:
#Shortened_training is what was actually used for finetuning GPT-3
write_new_dataset("shortened_training_0", validation)
write_new_dataset("shortened_training", training[0:407])
write_new_dataset("shortened_validation",training[407:509])
shortened_training = get_dataset("shortened_training.jsonl")
shortened_training = get_dataset("shortened_validation.jsonl")

In [7]:
print_len_dataset("shortened_training.jsonl")
print_len_dataset("shortened_validation.jsonl")

shortened_training.jsonl has length 407

shortened_validation.jsonl has length 102

